# Optimization
- 用於求非線性規劃問題的解，將問題表述為若干個變量的 純量函數(Scalar Function, e.g. P(x,y) Q(x, y))的最小值
- [scipy.optimize](https://docs.scipy.org/doc/scipy/tutorial/optimize.html)

## Q1

![function_1](pictures/fn1.png)

In [1]:
# ref: https://0809zheng.github.io/2021/08/23/minimize.html
from scipy.optimize import minimize
import numpy as np

# minimize a function
def fun(x):
    return x+1/x

x0 = np.array([2])
res = minimize(fun, x0, method='SLSQP')
# 輸出最小函數值和對應的 x 值
print(f"最小函數值: {res.fun}")
print(f"對應的 x 值: {res.x[0]}")

最小函數值: 2.0000000815356342
對應的 x 值: 1.000285585222987


![function_2](pictures/fn2.png)

In [6]:
from scipy.optimize import minimize
import numpy as np

# minimize a function with constraints
def fn(x):
    return (2+x[0])/(1+x[1])-3*x[0]+4*x[2]

# initial guess
x0 = np.array([0.5,0.5,0.5])

# constraints
# {Constraint, dict} or List of {Constraint, dict}, optional：約束條件，僅在SLSQP, COBYLA, trust-constr中使用
# 约束以字典的形式给出，其keys包含：
# type: str：约束类型。"eq" 等於0, "ineq" 大於等於 0
# fun: callable：约束函数, 限制在 [0.1, 0.9] 之間
cons = [{'type':'ineq', 'fun':lambda x:x[0]-0.1},
        {'type':'ineq', 'fun':lambda x:-x[0]+0.9},
        {'type':'ineq', 'fun':lambda x:x[1]-0.1},
        {'type':'ineq', 'fun':lambda x:-x[1]+0.9},
        {'type':'ineq', 'fun':lambda x:x[2]-0.1},
        {'type':'ineq', 'fun':lambda x:-x[2]+0.9}]

# minimize
res = minimize(fn, x0, method='SLSQP', constraints=cons)
print(res.fun)
print(res.x)

-0.773684210526435
[0.9 0.9 0.1]


![function_3](pictures/fn3.png)

In [7]:
from scipy.optimize import minimize
import numpy as np

def fn(x):
    return np.log2(1+x[0]*2/3)+np.log2(1+x[1]*3/4)

x0 = np.array([0.5,0.5])

cons = [{'type':'ineq', 'fun':lambda x:np.log2(1+x[0]*2/5)-5},
        {'type':'ineq', 'fun':lambda x:np.log2(1+x[1]*3/2)-5}]

res = minimize(fn, x0, method='SLSQP', constraints=cons)
print(res.fun)
print(res.x)

9.763212360886708
[77.5        20.66666658]


In [11]:
# ref: https://web.ntnu.edu.tw/~tsungwu/Python_DevOps/Part1_Basics&Math/section3_optimization.html
# 單變數: minimize_scalar
# 極大值的解，根據scipy說明文件，須把函數取負值 (sign=-1) 來找極大值
def f(x, sign=-1):
        return sign*(2*x**3+3*x**2-12*x-7)

from scipy.optimize import minimize_scalar
Result = minimize_scalar(f)
print(Result.x)

# 極大值: 須把負號加回來
print(-Result.fun)

-1.999999999777818
13.0


In [13]:
# 多變數: minimize
# 求相對極小值
def f(x, sign=1):
    x1 = x[0]
    x2 = x[1]
    return sign*(x1**3-4*x1*x2 +2*x2**2)

x0=[1,1]
Result = minimize(f, x0)

print(Result.x)
print(Result.fun)

[1.33333404 1.3333353 ]
-1.1851851851810147


## Q4: Advertising-Mix Problem
- ref: https://www.youtube.com/watch?v=d_D50ENWWKg&list=PLdYmuqrR3BQbsvmzv3ydqhz5OkRFHcWm5&index=19&ab_channel=Chih-huaHsu

![ad-mix_1](pictures/ad-mix1.png)
![ad-mix_2](pictures/ad-mix2.png)

In [3]:
from scipy.optimize import linprog

# Coefficients for the objective function (negative for maximization)
c = [-1000, -600]  # Maximize 1000 * TV + 600 * M

# Coefficients for the inequality constraints
A = [
    [300, 150],  # Ad Budget constraint: 300⋅TV+150⋅M ≤ 3600
    [90, 30],    # Planning Cost constraint: 90⋅TV+30⋅M ≤ 1200
]

# Right-hand side of inequality constraints
b = [3600, 1200]

# Bounds for the decision variables
x_bounds = (0, None)  # TV >= 0
y_bounds = (0, None)  # M >= 0

# Solve the linear programming problem
result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method="highs")

# Display results
if result.success:
    print(f"Optimal solution found:")
    print(f"Number of TV commercials (TV): {result.x[0]:.2f}")
    print(f"Number of magazine ads (M): {result.x[1]:.2f}")
    print(f"Maximum exposure: {-result.fun:.2f}")  # Negate the result for maximization
else:
    print("Optimization failed:", result.message)


Optimal solution found:
Number of TV commercials (TV): 0.00
Number of magazine ads (M): 24.00
Maximum exposure: 14400.00


## Q5: dual problem

![primal](pictures/primal.png)
![dual](pictures/dual.png)

In [1]:
from scipy.optimize import linprog

# Step 1: Define the coefficients of the objective function (negative for maximization)
c = [-8, -6]  # Coefficients of the dual objective (maximize 8y1 + 6y2, hence negative)

# Step 2: Define the inequality constraints (G and h in the form Gx <= h)
A = [
    [2, 1],   # Coefficients for y1 and y2 in the first constraint
    [1, 1]    # Coefficients for y1 and y2 in the second constraint
]
b = [3, 2]  # Right-hand side of the inequality constraints

# Step 3: Define the bounds for the variables (y1 >= 0, y2 >= 0)
bounds = [(0, None), (0, None)]  # Both y1 and y2 must be non-negative

# Step 4: Solve the linear programming problem
result = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method="highs")

# Step 5: Display the results
if result.success:
    print("Optimal solution found:")
    print(f"Optimal value: {-result.fun:.2f}")  # Negate to match maximization
    print(f"y1: {result.x[0]:.2f}")
    print(f"y2: {result.x[1]:.2f}")
else:
    print("Optimization failed:", result.message)


Optimal solution found:
Optimal value: 14.00
y1: 1.00
y2: 1.00


## Q6: Integer programming

![ip1](pictures/ip1.png)
![ip2](pictures/ip2.png)

In [1]:
from scipy.optimize import linprog

# Step 1: Define the weights matrix (w_ij)
weights = [
    [1, 3, 5],  # Scores for man 1
    [4, 2, 2],  # Scores for man 2
    [1, 5, 3]   # Scores for man 3
]

# Step 2: Flatten the weights matrix into a vector for the objective function
c = [-w for row in weights for w in row]  # Negative for maximization

# Step 3: Define the equality constraints
# Each man matches with exactly one woman
A_eq = []
for i in range(3):
    row = [0] * 9
    row[i * 3:(i + 1) * 3] = [1, 1, 1]
    A_eq.append(row)

# Each woman matches with exactly one man
for j in range(3):
    row = [0] * 9
    for i in range(3):
        row[i * 3 + j] = 1
    A_eq.append(row)

b_eq = [1] * 6  # Every man and woman matches exactly once

# Step 4: Define bounds for decision variables (0 <= x_ij <= 1)
bounds = [(0, 1) for _ in range(9)]

# Step 5: Solve the linear programming problem
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Step 6: Display the results
if result.success:
    print("Optimal solution found:")
    x_values = result.x.reshape((3, 3))  # Reshape back to 3x3 for clarity
    optimal_value = -result.fun  # Negate to match maximization
    print(f"Optimal value (maximum compatibility): {optimal_value}")
    print("Matchings:")
    for i in range(3):
        for j in range(3):
            if round(x_values[i][j]) == 1:  # Binary variable rounding
                print(f"Man {i+1} matched with Woman {j+1}")
else:
    print("Optimization failed:", result.message)


Optimal solution found:
Optimal value (maximum compatibility): 14.0
Matchings:
Man 1 matched with Woman 3
Man 2 matched with Woman 1
Man 3 matched with Woman 2
